In [16]:
import pandas as pd
from glob import glob
import os
from shutil import copyfile
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet
# from torchvision.models.inception import inception_v3
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.multiprocessing.set_sharing_strategy('file_system')
import pickle
from EarlyStopping import EarlyStopping
is_cuda = torch.cuda.is_available()
is_cuda

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook Untitled.ipynb to script
[NbConvertApp] Writing 2292 bytes to Untitled.py


True

In [2]:
class FeatureDataset(Dataset):
    def __init__(self,featlst1, featlst2, featlst3, labellst):
        self.featlst1 = featlst1
        self.featlst2 = featlst2
        self.featlst3 = featlst3
        self.labellst = labellst
        
    def __getitem__(self,index):
        return (self.featlst1[index], self.featlst2[index], self.featlst3[index], self.labellst[index])
    def __len__(self):
        return len(self.labellst)

#def

In [3]:
class LayerActivations():
    features=[]
    
    def __init__(self,model):
        self.features = []
        self.hook = model.register_forward_hook(self.hook_fn)
    
    def hook_fn(self, module, input, output):
        self.features.extend(output.view(output.size(0),-1))
    def remove(self):
        self.hook.remove()

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model1 = EfficientNet.from_pretrained('efficientnet-b1', num_classes=3).to(device)
# model1 = EfficientNet.from_pretrained('efficientnet-b1', num_classes=3).to(device)
model2 = EfficientNet.from_pretrained('efficientnet-b3', num_classes=3).to(device)
model3 = EfficientNet.from_pretrained('efficientnet-b4', num_classes=3).to(device)
model1 = model1.cuda()
model2 = model2.cuda()
model3 = model3.cuda()

# model1 = nn.Sequential(*list(model1.children())[:-1])
for p in model1.parameters():
    p.requires_grad = False
model2.aux_logits = False
for p in model2.parameters():
    p.requires_grad = False
for p in model3.parameters():
    p.requires_grad = False
    
    # 하이퍼 파라미터 * 훈련
INPUT_SIZE = 128
BATCH_SIZE = 128
NUM_CLASSES = 3
NUM_EPOCH = 50
PATIENCE = 10
MOMENTUM = 0.9
LEARNING_RATE = 0.005
WEIGHT_DECAY = 1e-04
PADDING = True
INBALANCE = False
AUG_FALLDOWN = False

loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model1.parameters(), lr=0.001, momentum=MOMENTUM, weight_decay= WEIGHT_DECAY)
optimizer = torch.optim.SGD(model2.parameters(), lr=0.001, momentum=MOMENTUM, weight_decay= WEIGHT_DECAY)
optimizer = torch.optim.SGD(model3.parameters(), lr=0.001, momentum=MOMENTUM, weight_decay= WEIGHT_DECAY)

Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b4


In [59]:
data_transform_trn = transforms.Compose([
    transforms.Resize((128,128)),
#     transforms.ColorJitter(brightness=0.2, hue=0.15, contrast=0.15),
    transforms.RandomHorizontalFlip(p=0.3),
#     transforms.RandomRotation(degrees=(-0.2, 0.2)),
    transforms.ToTensor(),
])
data_transform_val = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])


train_dest = ImageFolder('/home/mhkim/AGC/patch_image/labeled/no_padding/train', transform = data_transform_trn)
val_dest = ImageFolder('/home/mhkim/AGC/patch_image/labeled/no_padding/valid', transform=data_transform_val)
test_dest = ImageFolder("/home/mhkim/AGC/val",transform=data_transform_val)
classes = 3
train_loader = DataLoader(train_dest,batch_size=128, shuffle=False, num_workers=3)
val_loader = DataLoader(val_dest,batch_size=128, shuffle=False, num_workers=3)
test_loader = DataLoader(test_dest,batch_size=128, shuffle=False, num_workers=3)

In [60]:
import resource
rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (2048, rlimit[1]))

___###input the features of models###___

In [61]:
GPU_ID = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)
use_cuda = torch.cuda.is_available()
trn_labels = []
trn_features = []
for d, la in train_loader:
#     o = model1(Variable(d))
#     o = o.view(o.size(0),-1)
#     trn_labels.extend(la)
#     trn_features.extend(o)

    o = model1.extract_features(Variable(d.cuda()))
#     print(o.size())
    o = o.view(o.size(0),-1)
#     print(o.size())
#     break
    trn_labels.extend(la)
    trn_features.extend(o.data)
    
val_labels = []
val_features = []
for d,la in val_loader:
    o= model1.extract_features(Variable(d.cuda()))
    o= o.view(o.size(0),-1)
    #o = [batch_size,class_num]
    val_labels.extend(la)
    val_features.extend(o)
print("first model loader done")


first model loader done


In [62]:

#MODEL2
#trn_2_features = LayerActivations(model2.extract_features)
trn_2_features = []
for da, la in train_loader:
    o = model2.extract_features(Variable(da.cuda()))
    o = o.view(o.size(0),-1)
    trn_2_features.extend(o)

val_2_features = []
for da, la in val_loader:
    o = model2.extract_features(Variable(da.cuda()))
    o = o.view(o.size(0),-1)
    val_2_features.extend(o)
print("sec model loader done")


sec model loader done


In [63]:

#MODEL3
trn_3_features = []
for d, la in train_loader:
    o = model3.extract_features(Variable(d.cuda()))
    o = o.view(o.size(0),-1)
    trn_3_features.extend(o)

val_3_features = []
for d,la in val_loader:
    o= model3.extract_features(Variable(d.cuda()))
    o=o.view(o.size(0),-1)
    val_3_features.extend(o)
    
print("third model loader done")

third model loader done


In [64]:
from multiprocessing import Process
test_labels=[]
test_features = []
for d,la in test_loader:
    o= model1.extract_features(Variable(d.cuda()))
    o= o.view(o.size(0),-1)
    test_labels.extend(la)
    test_features.extend(o)
    
test2_features = []
for d,la in test_loader:
    o= model2.extract_features(Variable(d.cuda()))
    o= o.view(o.size(0),-1)
    test2_features.extend(o)
    
test3_features = []
for d,la in test_loader:
    o= model3.extract_features(Variable(d.cuda()))
    o= o.view(o.size(0),-1)
    test3_features.extend(o)

In [65]:
trn_feat_dset =FeatureDataset(trn_features, trn_2_features, trn_3_features, trn_labels)
val_feat_dset =FeatureDataset(val_features, val_2_features, val_3_features, val_labels)
test_feat_dset =FeatureDataset(test_features, test2_features, test3_features, test_labels)

trn_feat_loader = DataLoader(trn_feat_dset, batch_size=64, shuffle=True)
val_feat_loader = DataLoader(val_feat_dset, batch_size=64)
test_feat_loader = DataLoader(test_feat_dset, batch_size=64)

In [66]:
class Ensemble(nn.Module):
    def __init__(self,out_size,training=True):
        super().__init__()
        self.fc1 = nn.Linear(20480 ,256)
        self.fc2 = nn.Linear(24576, 256)
        self.fc3 = nn.Linear(28672 , 256)
        self.fc4 = nn.Linear(256,out_size)
        
    def forward(self, inp1, inp2, inp3):
        out1 = self.fc1(F.dropout(inp1,training=self.training))
        out2 = self.fc2(F.dropout(inp2,training=self.training))
        out3 = self.fc3(F.dropout(inp3,training=self.training))
        out = out1+ out2 + out3
        out = self.fc4(F.dropout(out,training=self.training))
        return out
    

In [67]:
import os
import torch
import shutil
def save_checkpoint(state, is_best, checkpoint='/home/mhkim/AGC/checkpoints', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    print(filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))
    

In [68]:
# early_stopping = EarlyStopping(patience=20, verbose=True)
is_best, best_acc =0 ,0
def fit(epoch, model, data_loader, phase='training', volatile=False):
    global is_best, best_acc
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct =0
    for batch_idx, (data1,data2,data3,target) in enumerate(data_loader):
#         print(data1,target)
        if is_cuda:
            data1,data2,data3,target = data1.cuda(),data2.cuda(),data3.cuda(),target.cuda()
            #data1,data2,data3,target = Variable(data1,volatile), Variable(data2,volatile), Variable(data3,volatile),Variable(target)
            if phase == 'training':
                optimizer.zero_grad()
            output = model(data1, data2, data3)
            loss = F.cross_entropy(output,target)
            running_loss += F.cross_entropy(output, target, size_average=False).data
            preds = output.data.max(dim=1, keepdim=True)[1]
            running_correct += preds.eq(target.data.view_as(preds)).sum()
            if phase == 'training':
                loss.backward()
                optimizer.step()

    accuracy = float(running_correct)/float(len(data_loader.dataset))
    loss = running_loss/len(data_loader.dataset)
    if phase == 'validation':
    #     print(accuracy)
        is_best = accuracy > best_acc
    #     print(is_best)
        if is_best : best_acc = accuracy
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'best_acc': accuracy,
            'optimizer': optimizer.state_dict(),
        }, is_best)
#         early_stopping(loss, model)
#         if early_stopping.early_stop:
#             print("Early stopping")
#             loss = None
       
    #if phase =='validation':
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}')
    return loss, accuracy

In [69]:
train_losses, train_acc = [],[]
val_losses, val_acc =[],[]
best_acc =.0
em = Ensemble(3)
if is_cuda:
    em = em.cuda()
optimizer = optim.Adam(em.parameters(),lr=0.0004)

for epoch in range(1,500):
    epoch_loss, epoch_acc = fit(epoch,em,trn_feat_loader,phase='training')
    val_epoch_loss, val_epoch_acc = fit(epoch,em,val_feat_loader, phase='validation')
    if(epoch_loss is None or val_epoch_loss is None):
        break
    train_losses.append(epoch_loss)
    train_acc.append(epoch_acc)
    val_losses.append(val_epoch_loss)
    val_acc.append(val_epoch_acc)



training loss is  0.44 and training accuracy is 20963/23777
/home/mhkim/AGC/checkpoints/checkpoint.pth.tar
validation loss is  0.41 and validation accuracy is 3532/4132
training loss is  0.11 and training accuracy is 22865/23777
/home/mhkim/AGC/checkpoints/checkpoint.pth.tar
validation loss is  0.46 and validation accuracy is 3559/4132
training loss is 0.076 and training accuracy is 23107/23777
/home/mhkim/AGC/checkpoints/checkpoint.pth.tar
validation loss is  0.57 and validation accuracy is 3600/4132
training loss is 0.071 and training accuracy is 23174/23777
/home/mhkim/AGC/checkpoints/checkpoint.pth.tar
validation loss is  0.54 and validation accuracy is 3581/4132
training loss is 0.067 and training accuracy is 23225/23777
/home/mhkim/AGC/checkpoints/checkpoint.pth.tar
validation loss is  0.66 and validation accuracy is 3577/4132
training loss is 0.062 and training accuracy is 23261/23777
/home/mhkim/AGC/checkpoints/checkpoint.pth.tar
validation loss is  0.59 and validation accuracy

____test____

In [70]:
def softmax(a) :
    a=a.detach().cpu().numpy()
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / float(sum_exp_a)
    y = torch.from_numpy(y)
    
    return y

def test_fit(epoch, model, data_loader, volatile=False):
    model.eval()
    volatile=True
    running_loss = 0.0
    running_correct =0
    for batch_idx, (data1,data2,data3,target) in enumerate(data_loader):
#         print(data1,target)
        if is_cuda:
            data1,data2,data3,target = data1.cuda(),data2.cuda(),data3.cuda(),target.cuda()
            #data1,data2,data3,target = Variable(data1,volatile), Variable(data2,volatile), Variable(data3,volatile),Variable(target)
            output = model(data1, data2, data3)
            output = softmax(output)
            print(output)
            loss = F.cross_entropy(output,target)
            
            running_loss += F.cross_entropy(output, target, size_average=False).data
            preds = output.data.max(dim=1, keepdim=True)[1]
            
            #running_correct += preds.eq(target. data.view_as(preds)).cpu().sum()
            running_correct += preds.eq(target.data.view_as(preds)).sum()
    loss = running_loss/len(data_loader.dataset)
    accuracy = running_correct/len(data_loader.dataset)
    #if phase =='validation':
    print(f'test loss is {loss:{5}.{2}} and test accuracy is {running_correct}/{len(data_loader.dataset)}')
    return loss, accuracy
epoch_losst, epoch_acct = test_fit(epoch,em,val_feat_loader)
print(epoch_acct)

tensor([[0., 0., nan],
        [0., 0., nan],
        [0., nan, nan],
        [0., nan, nan],
        [0., nan, 0.],
        [0., 0., nan],
        [0., nan, 0.],
        [0., nan, nan],
        [0., 0., 0.],
        [0., 0., nan],
        [0., nan, nan],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., nan, 0.],
        [0., nan, 0.],
        [0., nan, 0.],
        [0., 0., 0.],
        [0., nan, nan],
        [0., nan, 0.],
        [0., 0., nan],
        [0., 0., nan],
        [0., nan, nan],
        [0., nan, 0.],
        [0., nan, nan],
        [0., 0., 0.],
        [0., nan, nan],
        [0., 0., 0.],
        [0., nan, nan],
        [0., 0., nan],
        [0., nan, nan],
        [0., 0., nan],
        [0., 0., 0.],
        [0., 0., nan],
        [0., 0., nan],
        [0., 0., nan],
        [0., 0., nan],
        [0., 0., nan],
        [0., 0., nan],
        [0., 0., nan],
        [0., nan, nan],
        [0., 0., nan],
        [0., nan, 0.],
        [0., nan, 0.],
       

<ipython-input-70-eedbe01c9309>:3: RuntimeWarning: overflow encountered in exp
  exp_a = np.exp(a)
<ipython-input-70-eedbe01c9309>:5: RuntimeWarning: invalid value encountered in true_divide
  y = exp_a / float(sum_exp_a)


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _thnn_nll_loss_forward